In [70]:
# to export to latex use "jupyter nbconvert --to latex file.ipynb"

from IPython.display import display, Latex, Markdown


def func_Runge(x: float): return round((1 + x ** 2) ** -1, 3)


def get_default_nodes(n: int):
    h = 2 / n
    nodes_x = [round(-1 + k * h, 3) for k in range(n + 1)]
    nodes_f = [func_Runge(x) for x in nodes_x]
    return nodes_x, nodes_f


def d2f(x): return (6 * pow(x, 2) - 2) / pow(pow(x, 2) + 1, 3)

def df(x): return -2 * x / pow(1 + x ** 2, 2)


def phi1(t): return ((1 - t) ** 2) * (1 + 2 * t)


def phi2(t): return (t ** 2) * (3 - 2 * t)


def phi3(t): return t * (1 - t) ** 2


def phi4(t): return (t ** 2) * (t - 1)


def tma(A: list, B: list, C: list, F: list) -> list:
    alpha = [-C[0] / B[0]]
    beta = [F[0] / B[0]]

    for a, b, c, f in zip(A[1:], B[1:], C[1:], F[1:]):
        beta.append((f - a * beta[-1]) / (a * alpha[-1] + b))
        alpha.append(-c / (a * alpha[-1] + b))

    X = [(F[-1] - A[-1] * beta[-1]) / (B[-1] + A[-1] * alpha[-1])]
    alpha.reverse()
    beta.reverse()

    for a, b in zip(alpha[1:], beta[1:]):  # possible mistake
        X.append(a * X[-1] + b)

    X.reverse()
    return X


def spline1I(x: float, nodes_x: list, nodes_f: list) -> float:  # 5.a)
    l = 0
    for i in range(len(nodes_x) - 1):
        if nodes_x[i] < x <= nodes_x[i + 1]:
            l = i
            break

    h = [nodes_x[i + 1] - nodes_x[i] for i in range(len(nodes_x) - 1)]
    mu = [h[i-1] * pow(h[i-1] + h[i], -1) for i in range(1, len(h))]
    lmbda = [1 - x for x in mu]

    c = [3 * (mu[i] * ((nodes_f[i + 1] - nodes_f[i]) / h[i]))
         + lmbda[i] * (nodes_f[i] - nodes_f[i - 1]) / h[i - 1]
         for i in range(1, len(h) - 1)]

    mu.insert(0, 1)
    lmbda.append(0)
    c.insert(0, 2 * df(nodes_x[0]))
    c.append(2 * df(nodes_x[-1]))
    m = tma(lmbda, [2 for i in range(len(c))], mu, c)
    t = (x - nodes_x[l]) / h[l]
    m.insert(0, (c[0] - mu[0] * m[0]) / 2)

    return phi1(t) * nodes_f[l] + phi2(t) * nodes_f[l + 1] \
           + m[l] * h[l] * phi3(t) + \
           m[l + 1] * h[l] * phi4(t)


def spline1II(x: float, nodes_x: list, nodes_f: list) -> float:  # 5.б)
    l = 0
    for i in range(len(nodes_x) - 1):
        if nodes_x[i] < x <= nodes_x[i + 1]:
            l = i
            break

    h = [nodes_x[i + 1] - nodes_x[i] for i in range(len(nodes_x) - 1)]
    mu = [h[i-1] * pow(h[i-1] + h[i], -1) for i in range(1, len(h))]
    lmbda = [1 - x for x in mu]

    c = [3 * (mu[i] * ((nodes_f[i + 1] - nodes_f[i]) / h[i]))
         + lmbda[i] * (nodes_f[i] - nodes_f[i - 1]) / h[i - 1]
         for i in range(1, len(h) - 1)]

    mu.insert(0, 1)
    lmbda.append(1)
    c.insert(0, 3 * (nodes_f[1] - nodes_f[0]) / h[0] - h[0]
             * d2f(nodes_x[0]) / 2)
    c.append(3 * (nodes_f[-1] - nodes_f[-2]) / h[-2] - h[-2]
             * d2f(nodes_x[-1]) / 2)
    m = tma(lmbda, [2 for _ in range(len(c))], mu, c)
    t = (x - nodes_x[l]) / h[l]
    m.insert(0, (c[0] - mu[0] * m[0]) / 2)

    return phi1(t) * nodes_f[l] + phi2(t) * nodes_f[l + 1] \
           + m[l] * h[l] * phi3(t) + \
           m[l + 1] * h[l] * phi4(t)


def spline2I(x: float, nodes_x: list, nodes_f: list) -> float:  # 6.а)
    l = 0
    for i in range(len(nodes_x) - 1):
        if nodes_x[i] < x <= nodes_x[i + 1]:
            l = i
            break

    h = [nodes_x[i + 1] - nodes_x[i] for i in range(len(nodes_x) - 1)]
    mu = [h[i-1] * pow(h[i-1] + h[i], -1) for i in range(1, len(h))]
    lmbda = [1 - x for x in mu]

    d = [(6 / (h[i - 1] + h[i])) * ((nodes_f[i + 1] - nodes_f[i]) / h[i])
         -  (nodes_f[i] - nodes_f[i - 1])/ h[i - 1]
         for i in range(1, len(h) - 1)]

    lmbda.insert(0, 1)
    mu.append(1)
    d.insert(0, 6 * ((nodes_f[1] - nodes_f[0]) / h[0] - df(nodes_x[0])) / h[0])
    d.append(6 * (df(nodes_x[-1]) - (nodes_f[-1] - nodes_f[-2]) / h[-2]) / h[-2])
    M = tma(lmbda, [2 for _ in range(len(d))], mu, d)
    t = (x - nodes_x[l]) / h[l]
    M.insert(0, (d[0] - lmbda[0] * M[0]) / 2)

    return (1 - t) * nodes_f[l] + t * nodes_f[l + 1]\
           - (pow(h[l], 2) * t * (1 - t))\
           * ((2 - t) * M[l] + (1 + t) * M[l + 1])


def spline2II(x: float, nodes_x: list, nodes_f: list) -> float:  # 6.б)
    l = 0
    for i in range(len(nodes_x) - 1):
        if nodes_x[i] < x <= nodes_x[i + 1]:
            l = i
            break

    h = [nodes_x[i + 1] - nodes_x[i] for i in range(len(nodes_x) - 1)]
    mu = [h[i-1] * pow(h[i-1] + h[i], -1) for i in range(1, len(h))]
    lmbda = [1 - x for x in mu]

    d = [(6 / (h[i - 1] + h[i])) * ((nodes_f[i + 1] - nodes_f[i]) / h[i])
         -  (nodes_f[i] - nodes_f[i - 1])/ h[i - 1]
         for i in range(1, len(h) - 1)]

    lmbda.insert(0, 0)
    mu.append(0)
    d.insert(0, 2 * d2f(nodes_x[0]))
    d.append(2 * d2f(nodes_x[-1]))
    M = tma(lmbda, [2 for _ in range(len(d))], mu, d)
    t = (x - nodes_x[l]) / h[l]
    M.insert(0, (d[0] - lmbda[0] * M[0]) / 2)

    return (1 - t) * nodes_f[l] + t * nodes_f[l + 1] \
           - (pow(h[l], 2) * t * (1 - t)) * ((2 - t) * M[l] + (1 + t) * M[l + 1])


# Интерполирование сплайнами
Пусть $f(x)=\dfrac{1}{1+x^2},\:x\in\left[-1;1\right]$.<br>
Вычислить значения функции для $x=-1+\dfrac{h}{2},\:-\dfrac{h}{2},\:\dfrac{h}{2}$,
$\:1-\dfrac{h}{2},\:\left(h=\dfrac{2}{n},\:n=4,\:10, \:20,\:40\right)$, применяя
различные способы интерполирования:<br>
$5.$ Сплайн $S_{31}(f,x)$ с узлами $x_k=-1+kh,\:k=0,\ldots,n$, и параметрами
$m_i=S'(f,1x_i),i=0,\ldots,n,$ в котором<br>
$а).$ Граничные условия $I$ типа - $S^{'}(f,1)=f^{'}(1)$,
$S^{'}(f,-1)=f^{'}(-1);$<br>
$б).$ Граничные условия $II$ типа - $S^{''}(f,1)=f^{''}(1)$,
$S^{''}(f,-1)=f^{''}(-1).$<br>
$6.$ Сплайн $S_{31}(f,x)$ с узлами $x_k=-1+kh,\:k=0,\ldots,n$, и параметрами
$M_i=S'(f,1x_i),i=0,\ldots,n,$ в котором<br>
$а).$ Граничные условия $I$ типа - $S^{'}(f,1)=f^{'}(1)$,
$S^{'}(f,-1)=f^{'}(-1);$<br>
$б).$ Граничные условия $II$ типа - $S^{''}(f,1)=f^{''}(1)$,
$S^{''}(f,-1)=f^{''}(-1).$<br>


<b>№5 а)</b> Сплайн $S_{31}(f,x)$ с узлами $x_k=-1+kh,\:k=0,\ldots,n$, и параметрами
$m_i=S'(f,1x_i),i=0,\ldots,n,$ в котором граничные условия $I$ типа -
$S^{'}(f,1)=f^{'}(1)$,$S^{'}(f,-1)=f^{'}(-1);$

In [71]:
N = (4, 10, 20, 40)
Arguments = ((-1 + 1 / n, - 1 / n, 1 / n, 1 - 1 / n) for n in N)

for X, n in zip(Arguments, N):
    x_nodes, f_nodes = get_default_nodes(n)
    answers = []
    display(Latex("Случай, когда $n=%i$:"%n), Markdown("<br>"))
    for x in X:
        display(Latex("$S_{I}(f,%.3f)=%.4f$;"
                      %(x, spline1I(x, x_nodes, f_nodes))))
    display(Markdown("<br>"))

<IPython.core.display.Latex object>

<br>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<br>

<b>№5 б)</b> Сплайн $S_{31}(f,x)$ с узлами $x_k=-1+kh,\:k=0,\ldots,n$, и параметрами
$m_i=S'(f,1x_i),i=0,\ldots,n,$ в котором граничные условия $II$ типа -
$S^{''}(f,1)=f^{''}(1)$,$S^{''}(f,-1)=f^{''}(-1);$

In [72]:
N = (4, 10, 20, 40)
Arguments = ((-1 + 1 / n, - 1 / n, 1 / n, 1 - 1 / n) for n in N)

for X, n in zip(Arguments, N):
    x_nodes, f_nodes = get_default_nodes(n)
    answers = []
    display(Latex("Случай, когда $n=%i$:"%n), Markdown("<br>"))
    for x in X:
        display(Latex("$S_{II}(f,%.3f)=%.4f$;"
                      %(x, spline1II(x, x_nodes, f_nodes))))
    display(Markdown("<br>"))

<IPython.core.display.Latex object>

<br>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<br>

<b>№6 а)</b> Сплайн $S_{31}(f,x)$ с узлами $x_k=-1+kh,\:k=0,\ldots,n$, и параметрами
$M_i=S'(f,1x_i),i=0,\ldots,n,$ в котором граничные условия $I$ типа -
$S^{'}(f,1)=f^{'}(1)$,$S^{'}(f,-1)=f^{'}(-1);$

In [73]:
N = (4, 10, 20, 40)
Arguments = ((-1 + 1 / n, - 1 / n, 1 / n, 1 - 1 / n) for n in N)

for X, n in zip(Arguments, N):
    x_nodes, f_nodes = get_default_nodes(n)
    answers = []
    display(Latex("Случай, когда $n=%i$:"%n), Markdown("<br>"))
    for x in X:
        display(Latex("$S_{II}(f,%.3f)=%.4f$;"
                      %(x, spline2I(x, x_nodes, f_nodes))))
    display(Markdown("<br>"))

<IPython.core.display.Latex object>

<br>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<br>

<b>№6 б)</b> Сплайн $S_{31}(f,x)$ с узлами $x_k=-1+kh,\:k=0,\ldots,n$, и параметрами
$M_i=S'(f,1x_i),i=0,\ldots,n,$ в котором граничные условия $II$ типа -
$S^{''}(f,1)=f^{''}(1)$,$S^{''}(f,-1)=f^{''}(-1);$

In [74]:
N = (4, 10, 20, 40)
Arguments = ((-1 + 1 / n, - 1 / n, 1 / n, 1 - 1 / n) for n in N)

for X, n in zip(Arguments, N):
    x_nodes, f_nodes = get_default_nodes(n)
    answers = []
    display(Latex("Случай, когда $n=%i$:"%n), Markdown("<br>"))
    for x in X:
        display(Latex("$S_{II}(f,%.3f)=%.4f$;"
                      %(x, spline2II(x, x_nodes, f_nodes))))
    display(Markdown("<br>"))

<IPython.core.display.Latex object>

<br>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<br>

Вычисленные значения функции $f$ в точках:

In [75]:
N = (4, 10, 20, 40)
Arguments = ((-1 + 1 / n, - 1 / n, 1 / n, 1 - 1 / n) for n in N)
for X in Arguments:
    for x in X:
        display(Latex("$f(%.3f)=%.4f$;"%(x, func_Runge(x))))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>